# **FINANCE 361**

**Lecture 1**

## **Monte-Carlo Analysis of Stock Prices**

(The following Python code is non-examinable)

This notebook uses the Monte Carlo method to simulate the future price movements of selected stocks. The simulation illustrates the uncertainty inherent in financial markets, generating a wide range of potential outcomes based on historical price volatility and returns.

### **Preliminary Step #1**


*   Install the yfinance Python package
  *   To download market data from Yahoo! Finance API
  *   See: https://pypi.org/project/yfinance/


In [ ]:
pip install yfinance

### **Preliminary Step #2**



*   Import the required packages into Python for our simulations
  * Numpy:  https://numpy.org
  * Pandas: https://pandas.pydata.org
  * Matplotlib: https://matplotlib.org


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

np.random.seed(0)

### **Step 0: Define the parameters**

Defining our parameters now ensures that everything is consistent for the remainder of the script.

In [3]:
# Step 0: Define the parameters for the Monte Carlo simulation

NUM_SIMULATIONS = 50        # Defining the number of Monte Carlo simulations

NUM_DAYS_HISTORY = 252      # Approximate number of trading days in a year

NUM_DAYS_FORECAST = 132     # Approximate number of trading days in a month

### **Step 1: Retrieve historical stock data for Apple (AAPL)**

We'll retrieve 252 trading days, which is approximately 1 year.

In [ ]:
# Step 1: Retrieve historical stock data for Apple (AAPL)

ticker = 'AAPL'
data = yf.download(ticker, period=str(NUM_DAYS_HISTORY)+'d', interval='1d')

data

Step 1.1: Plotting the historical price data

In [ ]:
# Plot settings
fig, ax = plt.subplots(figsize=(14, 7))
plt.title('Monte Carlo Simulations of AAPL Stock Price (1 Year Historical & 6 Months Forecast)')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()

# Historical prices
historical_days = data.index[:]
historical_prices = data['Adj Close'][:]
ax.plot(historical_days, historical_prices, color='black', label='Historical Price')

# Forecast dates
last_day = historical_days[-1]
forecast_dates = pd.date_range(start=last_day, periods=NUM_DAYS_FORECAST + 1, closed='right')

ax.set_xlim([historical_days[0], forecast_dates[-1]])
ax.set_ylim([100, 300])

plt

### **Step 2: Calculate the daily returns of the stock**

In [7]:
# Step 2: Calculate the daily returns of the stock
daily_returns = np.log(data['Adj Close']/(data['Adj Close'].shift(1))).dropna()

# Calculate the mean and standard deviation of the daily returns
mean_daily_return = daily_returns.mean()
std_dev_daily_return = daily_returns.std()

# Calculate the mean and standard deviation of the annual returns
mean_annual_return = (1+daily_returns.mean())**252-1
std_dev_annual_return = daily_returns.std()*np.sqrt(252)

Step 2.1: Print the mean return and standard deviation values


In [ ]:
print(f"Mean daily return:                  {mean_daily_return:.2f}")
print(f"Daily standard deviation:           {std_dev_daily_return:.2f}")

print('\n')
print(f"Mean annual return:                 {mean_annual_return:.2f}")
print(f"Annual standard deviation:          {std_dev_annual_return:.2f}")

### **Step 3: Perform Monte Carlo simulation**


In [10]:
# Step 3: Perform Monte Carlo simulation

# Initialize an empty array to store the simulation results
monte_carlo_simulations = np.zeros((NUM_SIMULATIONS, NUM_DAYS_FORECAST))

# Generate the simulations
for n in range(NUM_SIMULATIONS):
    monte_carlo_simulations[n, 0] = data['Adj Close'].iloc[-1]
    for i in range(1, NUM_DAYS_FORECAST):
        random_daily_return = np.random.normal(mean_daily_return, std_dev_daily_return)
        monte_carlo_simulations[n, i] = monte_carlo_simulations[n, i-1] * (1 + random_daily_return)

### **Step 4: Plot the simulation results**

In [ ]:
# Step 4: Plot the simulation results
fig, ax = plt.subplots(figsize=(14, 7))
plt.title('Monte Carlo Simulations of AAPL Stock Price (1 Year Historical & 6 Months Forecast)')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()

# Historical prices
plt.plot(historical_days, historical_prices, color='black', label='Historical Price')

# Plot each simulation path
for n in range(NUM_SIMULATIONS):
    plt.plot(forecast_dates, monte_carlo_simulations[n, :], color='blue', alpha=0.1)

# Plotting the mean of the simulations
mean_simulation = np.mean(monte_carlo_simulations, axis=0)
plt.plot(forecast_dates, mean_simulation, color='red', linewidth=2, label='Mean Simulation')

ax.set_xlim([historical_days[0], forecast_dates[-1]])
ax.set_ylim([100, 300])

# Optionally, you can calculate the mean expected price at the end of the month
expected_price = np.mean(monte_carlo_simulations[:, -1])
print(f"Expected price at the end of 6 months: ${expected_price:.2f}")

### **Step 5: Next steps (in class)**

Extending the Monte Carlo simulation

*   What do we learn about the risk-return characteristics from our simulation?
  * How can we plot these characteristics intuitively?
*   What are the limitations/assumptions regarding the Monte-Carlo simulation?
*   How can we extend the simulation to multiple stocks, and what are the drawbacks of this approach?
  * I.e. Why do we need modern portfolio theory?



---

Additionally,

* What can we say from our simulations regarding behavioral finance?
* How does machine learning relate to our simulation model?



---



In [12]:
# Create a histogram of daily returns
counts, bins = np.histogram(daily_returns, bins=50)

In [13]:
# probability density function - Normal dist.
mu = mean_daily_return
sigma = std_dev_daily_return
x = bins
pdf = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(- (x - mu)**2 / (2 * sigma**2))

In [ ]:
# Create a Matplotlib plot
fig, ax = plt.subplots(figsize=(14, 7))
ax.hist(daily_returns, bins, density=True, alpha=0.6, color='b')
ax.plot(bins, pdf, linewidth=2, color='r')
plt.show()



---



In [15]:
aapl = yf.download('AAPL', period=str(NUM_DAYS_HISTORY)+'d', interval='1d')
gme = yf.download('GME', period=str(NUM_DAYS_HISTORY)+'d', interval='1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [16]:
df_portfolio = pd.DataFrame()
df_portfolio['AAPL Adj Close'] = aapl['Adj Close']
df_portfolio['GME Adj Close'] = gme['Adj Close']

In [ ]:
stocks_aapl = 1
stocks_gme = 5

df_portfolio['Portfolio'] = stocks_aapl*df_portfolio['AAPL Adj Close'] + stocks_gme*df_portfolio['GME Adj Close']
df_portfolio

In [ ]:
# Plot settings
fig, ax = plt.subplots(figsize=(14, 7))
plt.title('Monte Carlo Simulations of AAPL Stock Price (1 Year Historical & 6 Months Forecast)')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()

# Historical prices
historical_days = data.index[:]
historical_prices = df_portfolio['Portfolio'][:]
ax.plot(historical_days, historical_prices, color='black', label='Historical Price')

# Forecast dates
last_day = historical_days[-1]
forecast_dates = pd.date_range(start=last_day, periods=NUM_DAYS_FORECAST + 1, closed='right')

ax.set_xlim([historical_days[0], forecast_dates[-1]])
# ax.set_ylim([100, 300])

plt

In [ ]:
# Step 2: Calculate the daily returns of the stock
daily_returns = np.log(df_portfolio['Portfolio']/(df_portfolio['Portfolio'].shift(1))).dropna()

# Calculate the mean and standard deviation of the daily returns
mean_daily_return = daily_returns.mean()
std_dev_daily_return = daily_returns.std()

# Calculate the mean and standard deviation of the annual returns
mean_annual_return = (1+daily_returns.mean())**252-1
std_dev_annual_return = daily_returns.std()*np.sqrt(252)

print(f"Mean daily return:                  {mean_daily_return:.2f}")
print(f"Daily standard deviation:           {std_dev_daily_return:.2f}")

print('\n')
print(f"Mean annual return:                 {mean_annual_return:.2f}")
print(f"Annual standard deviation:          {std_dev_annual_return:.2f}")

In [20]:
# Step 3: Perform Monte Carlo simulation

# Initialize an empty array to store the simulation results
monte_carlo_simulations = np.zeros((NUM_SIMULATIONS, NUM_DAYS_FORECAST))

# Generate the simulations
for n in range(NUM_SIMULATIONS):
    monte_carlo_simulations[n, 0] = df_portfolio['Portfolio'].iloc[-1]
    for i in range(1, NUM_DAYS_FORECAST):
        random_daily_return = np.random.normal(mean_daily_return, std_dev_daily_return)
        monte_carlo_simulations[n, i] = monte_carlo_simulations[n, i-1] * (1 + random_daily_return)

In [ ]:
# Step 4: Plot the simulation results
fig, ax = plt.subplots(figsize=(14, 7))
plt.title('Monte Carlo Simulations of AAPL Stock Price (1 Year Historical & 6 Months Forecast)')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()

# Historical prices
plt.plot(historical_days, historical_prices, color='black', label='Historical Price')

# Plot each simulation path
for n in range(NUM_SIMULATIONS):
    plt.plot(forecast_dates, monte_carlo_simulations[n, :], color='blue', alpha=0.1)

# Plotting the mean of the simulations
mean_simulation = np.mean(monte_carlo_simulations, axis=0)
plt.plot(forecast_dates, mean_simulation, color='red', linewidth=2, label='Mean Simulation')

ax.set_xlim([historical_days[0], forecast_dates[-1]])
#ax.set_ylim([100, 300])

# Optionally, you can calculate the mean expected price at the end of the month
expected_price = np.mean(monte_carlo_simulations[:, -1])
print(f"Expected price at the end of 6 months: ${expected_price:.2f}")